In [1]:
from os import getenv

In [2]:
import airbyte_api as ab
import structlog
from airbyte_api import api, models
from airbyte_api.utils import BackoffStrategy, RetryConfig

## Structlog Config

In [3]:
structlog.configure(
    processors=[
        structlog.processors.add_log_level,
        structlog.processors.TimeStamper(fmt="iso"),
        structlog.processors.JSONRenderer(),
    ]
)

log = structlog.get_logger()

## Airbyte Credentials

In [4]:
api_url = getenv("AIRBYTE_API_URL", "http://localhost:8000/api/public/v1")

In [5]:
client_id = getenv("AIRBYTE_CLIENT_ID")

In [6]:
client_secret = getenv("AIRBYTE_CLIENT_SECRET")

## Airbyte API

In [7]:
client = ab.AirbyteAPI(
    server_url=api_url,
    security=models.Security(client_credentials=models.SchemeClientCredentials(
        client_id=client_id,
        client_secret=client_secret,
        token_url=api_url + "/applications/token"
    )),
    retry_config=RetryConfig(
        strategy="exponential-backoff",
        retry_connection_errors=True,
        backoff=BackoffStrategy(
            initial_interval=3,
            max_interval=30,
            max_elapsed_time=600,
            exponent=2
    ))
)

### List Connections

In [8]:
def list_connections():
    resp = client\
        .connections\
        .list_connections(api.ListConnectionsRequest(
            limit=2,
            offset=0,
            include_deleted=False
        ))

    return resp.connections_response.data

In [9]:
connections = list_connections()

### Get Sources

In [10]:
def get_source_by(id: int):
    res = client.sources.get_source(api.GetSourceRequest(source_id=id))
    return res.source_response

In [11]:
get_source_by(id=connections[0].source_id)

SourceResponse(configuration=SourceAirtable(credentials=None, SOURCE_TYPE=<SourceAirtableAirtable.AIRTABLE: 'airtable'>), created_at=1768422088, definition_id='0efee448-6948-49e2-b786-17db50647908', name='hackernews-rss-newest', source_id='d73fb160-9fbd-457e-891e-22d5ac5b5aee', source_type='rss', workspace_id='1a45e743-3de4-4652-adae-e5b2515668f5', resource_allocation=None)

### Get Destination

In [12]:
def get_destination_by(id: int):
    res = client.destinations.get_destination(api.GetDestinationRequest(destination_id=id))
    return res.destination_response

In [13]:
get_destination_by(id=connections[0].destination_id)

DestinationResponse(configuration=DestinationBigquery(dataset_id='hackernews_raw_data', dataset_location=<DatasetLocation.US_CENTRAL1: 'us-central1'>, project_id='iobruno-gcp-labs', cdc_deletion_mode=<CDCDeletionMode.HARD_DELETE: 'Hard delete'>, credentials_json='**********', DESTINATION_TYPE=<Bigquery.BIGQUERY: 'bigquery'>, disable_type_dedupe=False, loading_method=BatchedStandardInserts(additional_properties=None, method=<Method.STANDARD: 'Standard'>), raw_data_dataset=None), created_at=1768458307, definition_id='22f6c74f-5699-40ff-833c-4a879ea40133', destination_id='14e21b21-85d5-46b8-916a-df3c087ac4fe', destination_type='bigquery', name='bigquery-hackernews', workspace_id='1a45e743-3de4-4652-adae-e5b2515668f5', resource_allocation=None)

## Putting it all together

In [14]:
for idx, conn in enumerate(list_connections()):
    source = get_source_by(id=conn.source_id)
    destination = get_destination_by(id=conn.destination_id)
    print(f"conn{idx} = {conn.to_json()}\n")
    print(f"source{idx} = {source.to_json()}\n")
    print(f"destination{idx} = {destination.to_json()}\n")

conn0 = {"configurations": {"streams": [{"name": "items", "cursorField": ["published"], "includeFiles": false, "mappers": [], "primaryKey": [["guid"]], "selectedFields": [], "syncMode": "incremental_deduped_history"}]}, "connectionId": "b7715560-ec4e-44e8-8d64-830f6463e341", "createdAt": 1768460945, "destinationId": "14e21b21-85d5-46b8-916a-df3c087ac4fe", "name": "hackernews-rss-newest \u2192 bigquery-hackernews", "schedule": {"scheduleType": "manual"}, "sourceId": "d73fb160-9fbd-457e-891e-22d5ac5b5aee", "status": "active", "tags": [], "workspaceId": "1a45e743-3de4-4652-adae-e5b2515668f5", "namespaceDefinition": "destination", "nonBreakingSchemaUpdatesBehavior": "propagate_columns", "prefix": "newest_"}

source0 = {"configuration": {"sourceType": "airtable"}, "createdAt": 1768422088, "definitionId": "0efee448-6948-49e2-b786-17db50647908", "name": "hackernews-rss-newest", "sourceId": "d73fb160-9fbd-457e-891e-22d5ac5b5aee", "sourceType": "rss", "workspaceId": "1a45e743-3de4-4652-adae-e5b